## 성능 측정
LLama3 모델을 Inferentia2에 성공적으로 배포하였으니, 이제 이 모델의 성능을 벤치마킹하기 위해 코드를 다운로드하고 모델의 성능을 측정할 준비를 합니다. 다음 명령어를 통해 벤치마크를 실행할 수 있습니다.

In [22]:
!pip install -q nltk

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 33.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [25]:
!python benchmark.py \
  --endpoint http://127.0.0.1:8080/predictions/llama3 \
  --num-concurrent-requests 10 \
  --max-num-completed-requests 100 \
  --input-tokens 4096 \
  --max-new-tokens 4096 \
  --timeout 600 \
  --results-dir results

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|█████████████████████████████████████████| 100/100 [12:20<00:00,  7.41s/it]

Benchmark Results:
Concurrent requests: 10
Input tokens: 4096
Max new tokens: 4096
Avg. Latency: 61887.04ms
Avg. Time-to-First-Token (TTFT): 12997.58ms
Output Token Throughput: 1096.46 tokens/sec
Requests per second: 0.61
Total time: 162.87s
Total output tokens generated: 178586


이 명령은 동시에 10명의 사용자가 100개의 요청을 처리하면서, 모델의 첫 토큰까지의 시간, 토큰당 지연 시간(ms/token) 및 처리량(tokens/s)을 측정합니다. 이 모든 상세 결과는 결과 폴더에서 확인할 수 있습니다.

이제 결과를 파싱하고 표시해 보겠습니다. 다음은 Python 코드와 그 출력 결과입니다.

In [16]:
# 결과 폴더에서 summary.json 파일을 읽고 결과를 출력
import glob
import json

with open(glob.glob('results/*summary.json')[0], 'r') as file:
    results = json.load(file)

print("\nBenchmark Results:")
print(f"Concurrent requests: {results['num_concurrent_requests']}")
print(f"Input tokens: {results['input_tokens']}")
print(f"Max new tokens: {results['max_new_tokens']}")
print(f"Avg. Latency: {results['results']['avg_latency']:.2f}ms")
print(f"Avg. Time-to-First-Token (TTFT): {results['results']['avg_ttft']:.2f}ms")
print(f"Output Token Throughput: {results['results']['output_tokens_per_second']:.2f} tokens/sec")
print(f"Requests per second: {results['results']['requests_per_second']:.2f}")
print(f"Total time: {results['results']['total_time']:.2f}s")
print(f"Total output tokens generated: {results['results']['total_output_tokens']}")


Benchmark Results:
Avg. Input token length: 512.00
Avg. Output token length: 27.00


KeyError: 'avg_total_tokens'

In [3]:
# Concurrent requests: 25
# Avg. Input token length: 512.0
# Avg. Output token length: 27.0
# Avg. First-Time-To-Token: 469.89ms
# Avg. Throughput: 57.45 tokens/sec
# Avg. Latency: 28.02ms/token